In [1]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-ffdee303-0dd4-513d-b18c-beba028b49c7"
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from hydra.utils import instantiate
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import pandas as pd

device = "cuda:0"

# Load data and models

In [2]:
os.chdir("/allen/aics/modeling/ritvik/projects/benchmarking_representations/")
save_path = "./test_cellpack_save_embeddings/"

In [3]:
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint

from br.data.get_datamodules import get_data
from br.models.load_models import load_model_from_path
from br.models.save_embeddings import get_pc_loss


def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(
        dataset_name, results_path
    )  # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [31]:
dataset_name = "cellpack"
batch_size = 2
debug = True
results_path = (
    "/allen/aics/modeling/ritvik/projects/benchmarking_representations/br/configs/results/"
)
data_list, all_models, run_names, model_sizes = get_data_and_models(
    dataset_name, batch_size, results_path, debug
)
gg = pd.DataFrame()
gg['model'] = run_names
gg['model_size'] = model_sizes
gg.to_csv(save_path + 'model_sizes.csv')

# Compute embeddings and emissions

In [33]:
from br.models.save_embeddings import save_embeddings

debug = False
splits_list = ["train", 'val', "test"]
meta_key = "rule"
eval_scaled_img = [False] * 5
eval_scaled_img_params = [{}] * 5
loss_eval_list = None
sample_points_list = [True, True, False, False, False]
skew_scale = 100
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

Processing train


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [02:03<00:00,  4.30it/s]


Processing val


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:27<00:00,  4.13it/s]


Processing test


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 531/531 [00:30<00:00, 17.14it/s]


Processing val


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:06<00:00, 17.02it/s]


Processing test


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:06<00:00, 17.12it/s]


In [36]:
1

1

In [34]:
from br.models.save_embeddings import save_emissions

max_batches = 2
save_emissions(
    save_path,
    data_list,
    all_models,
    run_names,
    max_batches,
    debug,
    device,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

  2%|█▉                                                                                                              | 2/117 [00:02<02:40,  1.40s/it]


# Compute benchmarking features

In [35]:
from br.models.compute_features import compute_features
from br.models.save_embeddings import get_pc_loss_chamfer
from br.models.utils import get_all_configs_per_dataset

keys = ["pcloud"] * 5
max_embed_dim = 256
DATA_LIST = get_all_configs_per_dataset(results_path)
data_config_list = DATA_LIST[dataset_name]["data_paths"]

evolve_params = {
    "modality_list_evolve": keys,
    "config_list_evolve": data_config_list,
    "num_evolve_samples": 40,
    "compute_evolve_dataloaders": False,
    "eval_meshed_img": [False] * 5,
    "skew_scale": 100,
    "eval_meshed_img_model_type": [None] * 5,
    "only_embedding": False,
    "fit_pca": False,
}

loss_eval = get_pc_loss_chamfer()
# loss_eval_list = [torch.nn.MSELoss(reduction='none')]*2 + [loss_eval, loss_eval]
loss_eval_list = [loss_eval] * 5
use_sample_points_list = [True, True, False, False, False]

classification_params = {"class_labels": ["rule"]}
rot_inv_params = {"squeeze_2d": False, "id": "cell_id"}

regression_params = {"df_feat": None, "target_cols": None, "feature_df_path": None}

compactness_params = {
    "method": "mle",
    "num_PCs": None,
    "blobby_outlier_max_cc": None,
    "check_duplicates": True,
}

splits_list = ["train", "val", "test"]
compute_embeds = False

metric_list = [
    "Rotation Invariance Error",
    "Evolution Energy",
    "Reconstruction",
    "Classification",
    "Compactness",
]


compute_features(
    dataset=dataset_name,
    results_path=results_path,
    embeddings_path=save_path,
    save_folder=save_path,
    data_list=data_list,
    all_models=all_models,
    run_names=run_names,
    use_sample_points_list=use_sample_points_list,
    keys=keys,
    device=device,
    max_embed_dim=max_embed_dim,
    splits_list=splits_list,
    compute_embeds=compute_embeds,
    classification_params=classification_params,
    regression_params=regression_params,
    metric_list=metric_list,
    loss_eval_list=loss_eval_list,
    evolve_params=evolve_params,
    rot_inv_params=rot_inv_params,
    compactness_params=compactness_params,
)

Computing rotation invariance


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:09<00:00, 11.73it/s]


Getting reconstruction
Computing compactness


  0%|                                                                                                                          | 0/5 [00:00<?, ?it/s]

(1524, 256)


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.66it/s]


Outlier column is outlier
Computing classification


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:12<00:00,  2.45s/it]


Computing evolution


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:08<00:00, 13.04it/s]


# Polar plot viz

In [37]:
from br.features.plot import collect_outputs
from br.features.plot import plot

model_order = ["Classical_image", "SO3_image", "Classical_pointcloud", "SO3_pointcloud"]
metric_list = ['reconstruction', 'emissions', 'classification_rule', 
               'compactness', 'evolution_energy', 
               'model_sizes', 'rotation_invariance_error']
norm = 'std'
title = 'cellpack_comparison'
colors_list = None
unique_expressivity_metrics = ['Classification_rule']
df, df_non_agg = collect_outputs(save_path, norm, model_order, metric_list)
plot(save_path, df, model_order, title, colors_list, norm, unique_expressivity_metrics)

reconstruction
emissions
classification_rule
compactness
evolution_energy
model_sizes
rotation_invariance_error


# Latent walks

In [26]:
from br.models.compute_features import get_embeddings
from br.models.utils import get_all_configs_per_dataset
run_names = ['SO3_pointcloud_jitter']
DATASET_INFO = get_all_configs_per_dataset(results_path)
all_ret, df = get_embeddings(run_names, dataset_name, DATASET_INFO, save_path)

In [27]:
all_ret

,Unnamed: 0,CellId,mu_0,mu_1,mu_2,mu_3,mu_4,mu_5,mu_6,mu_7,...,shcoeffs_L9M5C_lcc,shcoeffs_L9M5S_lcc,shcoeffs_L9M6C_lcc,shcoeffs_L9M6S_lcc,shcoeffs_L9M7C_lcc,shcoeffs_L9M7S_lcc,shcoeffs_L9M8C_lcc,shcoeffs_L9M8S_lcc,shcoeffs_L9M9C_lcc,shcoeffs_L9M9S_lcc
0,0,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.063755,0.139987,0.617743,0.033245,0.081398,0.083860,0.026474,0.016656,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
1,1,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.066745,0.140154,0.618154,0.033134,0.094268,0.083241,0.025570,0.016725,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
2,2,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.068114,0.140243,0.618402,0.033210,0.091530,0.084005,0.024861,0.016814,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
3,3,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.066769,0.138308,0.617939,0.033198,0.085800,0.083618,0.025079,0.016598,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
4,4,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.065599,0.139665,0.617569,0.033296,0.090452,0.083318,0.025512,0.016669,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
5,5,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.066779,0.143080,0.617680,0.033203,0.096191,0.083517,0.026552,0.016820,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
6,6,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.067627,0.139652,0.617389,0.033489,0.091568,0.084269,0.027490,0.016777,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
7,7,9c1ff213-4e9e-4b73-a942-3baf9d37a50f_0,0.069063,0.139853,0.619011,0.033278,0.092649,0.083910,0.025870,0.016706,...,0.06163,0.019391,-0.103761,0.066481,-0.027658,0.023938,0.033112,-0.015316,0.061876,0.074256
